In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import os

# ----------------------------------
# Page Configuration
# ----------------------------------
st.set_page_config(
    page_title="Pricing & Inventory Optimization",
    layout="wide"
)

st.title("Pricing & Inventory Optimization Dashboard")
st.markdown("""
This dashboard evaluates pricing strategies by combining **demand response,
revenue impact, and inventory feasibility**.

It is designed for **executive decision-making** and operational planning.
""")

# ----------------------------------
# Robust Data Loading (PRODUCTION SAFE)
# ----------------------------------
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DATA_DIR = os.path.join(BASE_DIR, "..", "data")

scenario_df = pd.read_csv(os.path.join(DATA_DIR, "scenario_summary.csv"))
category_df = pd.read_csv(os.path.join(DATA_DIR, "category_decision_table.csv"))
elasticity_df = pd.read_csv(os.path.join(DATA_DIR, "elasticity_risk_table.csv"))

# ----------------------------------
# Scenario Selector
# ----------------------------------
st.subheader("Pricing Scenario Selection")

scenario = st.selectbox(
    "Select Pricing Scenario",
    scenario_df["Scenario"].unique()
)

selected = scenario_df[scenario_df["Scenario"] == scenario].iloc[0]

# ----------------------------------
# Executive KPI Section
# ----------------------------------
st.subheader("Executive KPIs")

col1, col2, col3 = st.columns(3)

col1.metric(
    "Feasible Revenue",
    f"${selected['Feasible_Revenue']:,.0f}"
)

col2.metric(
    "Theoretical Revenue",
    f"${selected['Theoretical_Revenue']:,.0f}"
)

col3.metric(
    "Stockout Risk",
    selected["Stockout_Risk"]
)

# ----------------------------------
# Revenue Comparison Chart
# ----------------------------------
st.subheader("Feasible Revenue by Pricing Scenario")

fig, ax = plt.subplots()
ax.bar(
    scenario_df["Scenario"],
    scenario_df["Feasible_Revenue"]
)
ax.set_ylabel("Revenue")
ax.set_title("Feasible Revenue Comparison")
ax.grid(axis="y", linestyle="--", alpha=0.6)

st.pyplot(fig)

# ----------------------------------
# Category-Level Strategy Table
# ----------------------------------
st.subheader("Category-Level Pricing Strategy")

st.dataframe(
    category_df,
    use_container_width=True
)

# ----------------------------------
# Elasticity vs Inventory Risk Visualization
# ----------------------------------
st.subheader("Elasticity vs Inventory Risk")

fig, ax = plt.subplots()

for band in elasticity_df["elasticity_band"].unique():
    subset = elasticity_df[elasticity_df["elasticity_band"] == band]
    ax.scatter(
        subset["price_elasticity_proxy"],
        subset["stock_minus_10"],
        label=band,
        alpha=0.8
    )

ax.set_xlabel("Price Elasticity (Proxy)")
ax.set_ylabel("Stockout Rate (-10% Price)")
ax.set_title("Elasticity vs Inventory Risk")
ax.legend()
ax.grid(True)

st.pyplot(fig)

# ----------------------------------
# Executive Recommendations
# ----------------------------------
st.subheader("Executive Recommendations")

st.markdown("""
### Key Insights
- Aggressive discounts create **high theoretical revenue** but introduce **significant stockout risk**.
- Inventory constraints materially reduce achievable revenue.
- Moderate pricing strategies provide **better execution stability**.

### Business Recommendation
> Optimize for **realized revenue**, not theoretical demand.
> Pricing decisions must be **coordinated with inventory readiness**.

This dashboard enables leaders to balance **growth, profitability, and operational feasibility**.
""")
